## Tarea#1

## BigData

### Juan Pablo Fernández Deltado

### Pre-requisitos

- findspark
- pyspark
- spark

El set de datos a cargar va ser el del titanic, usando las mayoría de sus columnas.

El primer paso es cargar las librerías y el contenido de las funcionalidades de pyspark

In [1]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.types import *
from datetime import datetime
from pyspark.sql.functions import col, date_format, udf 
from pyspark.sql.types import DateType

import findspark
findspark.init('C:\Spark')

spark=SparkSession.builder.appName('data_processing').getOrCreate()


Se carga el set de datos de titanic y sus columnas respectivas.

In [2]:
names_df = spark \
    .read \
    .format("csv") \
    .option("path", "titanic.csv") \
    .option("header", True) \
    .schema(StructType([
                StructField("PassengerId", IntegerType()),
                StructField("Survived", IntegerType()),
                StructField("Pclass", IntegerType()),
                StructField("Name", StringType()),
                StructField("Sex", StringType()),
    StructField("Age", IntegerType()),
    StructField("SibSp", IntegerType()),
    StructField("Parch", IntegerType()),
    StructField("Ticket", StringType()),
    StructField("Fare", IntegerType()),
    StructField("Cabin", StringType()),
                StructField("Embarked", StringType())])) \
    .load()

In [3]:
names_df.printSchema()
names_df.show()


root
 |-- PassengerId: integer (nullable = true)
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: integer (nullable = true)
 |-- Cabin: string (nullable = true)
 |-- Embarked: string (nullable = true)

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+----+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+----+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|  22|    1|    0|       A/5 21171|null| null|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female|  38|    1|    0|        PC 17

1) Se visualiza el esquema del DataFrame:

In [4]:
names_df.printSchema()

root
 |-- PassengerId: integer (nullable = true)
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: integer (nullable = true)
 |-- Cabin: string (nullable = true)
 |-- Embarked: string (nullable = true)



2) Se visualiza el DataFrame:

In [5]:
names_df.show()

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+----+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+----+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|  22|    1|    0|       A/5 21171|null| null|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female|  38|    1|    0|        PC 17599|null|  C85|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female|  26|    0|    0|STON/O2. 3101282|null| null|       S|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|  35|    1|    0|          113803|null| C123|       S|
|          5|       0|     3|Allen, Mr. Willia...|  male|  35|    0|    0|          373450|null| null|       S|
|          6|       0|     3|    Moran, Mr. James|  male|null|    0|    0|          330877|null| null|  

In [10]:
names_df.fillna('0').show()

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+----+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+----+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|  22|    1|    0|       A/5 21171|null|    0|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female|  38|    1|    0|        PC 17599|null|  C85|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female|  26|    0|    0|STON/O2. 3101282|null|    0|       S|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|  35|    1|    0|          113803|null| C123|       S|
|          5|       0|     3|Allen, Mr. Willia...|  male|  35|    0|    0|          373450|null|    0|       S|
|          6|       0|     3|    Moran, Mr. James|  male|null|    0|    0|          330877|null|    0|  

Se rellenan los null con parámetros específicos por columna, de nuevo se muestran no se modifican:

4) Se descartan los que tengan algun valor null:

In [11]:
names_df.na.drop().show()

+-----------+--------+------+--------------------+------+---+-----+-----+------+----+-----------+--------+
|PassengerId|Survived|Pclass|                Name|   Sex|Age|SibSp|Parch|Ticket|Fare|      Cabin|Embarked|
+-----------+--------+------+--------------------+------+---+-----+-----+------+----+-----------+--------+
|         22|       1|     2|Beesley, Mr. Lawr...|  male| 34|    0|    0|248698|  13|        D56|       S|
|         28|       0|     1|Fortune, Mr. Char...|  male| 19|    3|    2| 19950| 263|C23 C25 C27|       S|
|         89|       1|     1|Fortune, Miss. Ma...|female| 23|    3|    2| 19950| 263|C23 C25 C27|       S|
|        111|       0|     1|Porter, Mr. Walte...|  male| 47|    0|    0|110465|  52|       C110|       S|
|        184|       1|     2|Becker, Master. R...|  male|  1|    2|    1|230136|  39|         F4|       S|
|        194|       1|     2|Navratil, Master....|  male|  3|    1|    1|230080|  26|         F2|       S|
|        210|       1|     1|    Blan

5) Se descartan los que tengan algun valor null en names_df:

In [6]:
names_df.na.drop(subset='Age').show()

+-----------+--------+------+--------------------+------+---+-----+-----+----------------+----+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex|Age|SibSp|Parch|          Ticket|Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+---+-----+-----+----------------+----+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male| 22|    1|    0|       A/5 21171|null| null|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female| 38|    1|    0|        PC 17599|null|  C85|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female| 26|    0|    0|STON/O2. 3101282|null| null|       S|
|          4|       1|     1|Futrelle, Mrs. Ja...|female| 35|    1|    0|          113803|null| C123|       S|
|          5|       0|     3|Allen, Mr. Willia...|  male| 35|    0|    0|          373450|null| null|       S|
|          7|       0|     1|McCarthy, Mr. Tim...|  male| 54|    0|    0|           17463|null|  E46|       S|
|

Se pueden reemplazar valores por otros...

In [20]:
names_df.replace("Mr.","MR.").show()

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+----+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+----+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|  22|    1|    0|       A/5 21171|null| null|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female|  38|    1|    0|        PC 17599|null|  C85|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female|  26|    0|    0|STON/O2. 3101282|null| null|       S|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|  35|    1|    0|          113803|null| C123|       S|
|          5|       0|     3|Allen, Mr. Willia...|  male|  35|    0|    0|          373450|null| null|       S|
|          6|       0|     3|    Moran, Mr. James|  male|null|    0|    0|          330877|null| null|  

Se pueden descartar columnas completas:

In [11]:
names_df.drop('Cabin').show()

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|Fare|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|  22|    1|    0|       A/5 21171|null|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female|  38|    1|    0|        PC 17599|null|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female|  26|    0|    0|STON/O2. 3101282|null|       S|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|  35|    1|    0|          113803|null|       S|
|          5|       0|     3|Allen, Mr. Willia...|  male|  35|    0|    0|          373450|null|       S|
|          6|       0|     3|    Moran, Mr. James|  male|null|    0|    0|          330877|null|       Q|
|          7|       0|     1|McCarthy, Mr. Tim

Se analizan los datos:

In [12]:
names_df.count()

891

In [13]:
len(names_df.columns)

12

In [14]:
names_df.show(3)

+-----------+--------+------+--------------------+------+---+-----+-----+----------------+----+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex|Age|SibSp|Parch|          Ticket|Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+---+-----+-----+----------------+----+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male| 22|    1|    0|       A/5 21171|null| null|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female| 38|    1|    0|        PC 17599|null|  C85|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female| 26|    0|    0|STON/O2. 3101282|null| null|       S|
+-----------+--------+------+--------------------+------+---+-----+-----+----------------+----+-----+--------+
only showing top 3 rows



In [15]:
names_df.summary().show()

+-------+-----------------+-------------------+------------------+--------------------+------+------------------+------------------+-------------------+------------------+-----------------+-----+--------+
|summary|      PassengerId|           Survived|            Pclass|                Name|   Sex|               Age|             SibSp|              Parch|            Ticket|             Fare|Cabin|Embarked|
+-------+-----------------+-------------------+------------------+--------------------+------+------------------+------------------+-------------------+------------------+-----------------+-----+--------+
|  count|              891|                891|               891|                 891|   891|               689|               891|                891|               891|              161|  204|     889|
|   mean|            446.0| 0.3838383838383838| 2.308641975308642|                null|  null|29.847605224963715|0.5230078563411896|0.38159371492704824|260318.54916792738|33.472049

Mostrar solo unas columnas y filas:

In [16]:
names_df.select(['Survived','Name','Age']).show()

+--------+--------------------+----+
|Survived|                Name| Age|
+--------+--------------------+----+
|       0|Braund, Mr. Owen ...|  22|
|       1|Cumings, Mrs. Joh...|  38|
|       1|Heikkinen, Miss. ...|  26|
|       1|Futrelle, Mrs. Ja...|  35|
|       0|Allen, Mr. Willia...|  35|
|       0|    Moran, Mr. James|null|
|       0|McCarthy, Mr. Tim...|  54|
|       0|Palsson, Master. ...|   2|
|       1|Johnson, Mrs. Osc...|  27|
|       1|Nasser, Mrs. Nich...|  14|
|       1|Sandstrom, Miss. ...|   4|
|       1|Bonnell, Miss. El...|  58|
|       0|Saundercock, Mr. ...|  20|
|       0|Andersson, Mr. An...|  39|
|       0|Vestrom, Miss. Hu...|  14|
|       1|Hewlett, Mrs. (Ma...|  55|
|       0|Rice, Master. Eugene|   2|
|       1|Williams, Mr. Cha...|null|
|       0|Vander Planke, Mr...|  31|
|       1|Masselmani, Mrs. ...|null|
+--------+--------------------+----+
only showing top 20 rows



Mostrar solo ciertos datos a partir de un criterio:

In [17]:
names_df.filter(names_df['Age'] > 50).count()

62

In [18]:
names_df.filter(names_df['Age'] > 50).show()

+-----------+--------+------+--------------------+------+---+-----+-----+-----------+----+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex|Age|SibSp|Parch|     Ticket|Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+---+-----+-----+-----------+----+-----+--------+
|          7|       0|     1|McCarthy, Mr. Tim...|  male| 54|    0|    0|      17463|null|  E46|       S|
|         12|       1|     1|Bonnell, Miss. El...|female| 58|    0|    0|     113783|null| C103|       S|
|         16|       1|     2|Hewlett, Mrs. (Ma...|female| 55|    0|    0|     248706|  16| null|       S|
|         34|       0|     2|Wheadon, Mr. Edwa...|  male| 66|    0|    0| C.A. 24579|null| null|       S|
|         55|       0|     1|Ostby, Mr. Engelh...|  male| 65|    0|    1|     113509|null|  B30|       C|
|         95|       0|     3|   Coxon, Mr. Daniel|  male| 59|    0|    0|     364500|null| null|       S|
|         97|       0|     1|Goldschmidt, Mr. 

Se puede saber cuantos datos hay de una caterogira especifica:

In [24]:
names_df.groupBy('Age').count().show()

+---+-----+
|Age|count|
+---+-----+
| 22|   27|
| 38|   11|
| 26|   18|
| 35|   18|
| 54|    8|
|  2|   10|
| 27|   18|
| 14|    6|
|  4|   10|
| 58|    5|
| 20|   15|
| 39|   14|
| 55|    2|
| 31|   17|
| 34|   15|
| 15|    5|
| 28|   25|
|  8|    4|
| 19|   25|
| 40|   13|
+---+-----+
only showing top 20 rows



También puedo mostrar los datos un orden ascendente o decendente

In [19]:
names_df.sort("Age").show()

+-----------+--------+------+--------------------+------+----+-----+-----+---------------+----+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|         Ticket|Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+---------------+----+-----+--------+
|         37|       1|     3|    Mamee, Mr. Hanna|  male|null|    0|    0|           2677|null| null|       C|
|         79|       1|     2|Caldwell, Master....|  male|null|    0|    2|         248738|  29| null|       S|
|          6|       0|     3|    Moran, Mr. James|  male|null|    0|    0|         330877|null| null|       Q|
|         46|       0|     3|Rogers, Mr. Willi...|  male|null|    0|    0|S.C./A.4. 23567|null| null|       S|
|         20|       1|     3|Masselmani, Mrs. ...|female|null|    0|    0|           2649|null| null|       C|
|         43|       0|     3| Kraeff, Mr. Theodor|  male|null|    0|    0|         349253|null| null|       C|
|